In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../data/processed/processed_data.csv')
df_2000 = df[df['Period'] == '2000s'].reset_index(drop=True)

In [4]:
df_2000.head()

,Date,U.S. 30 Day TBill TR,Russell 2000 TR,S&P 500 TR,LB LT Gvt/Credit TR,MSCI EAFE TR,Period
0,2002-01-02,0.1484,-1.105424,-1.557383,-2.2782,-5.783732,2000s
1,2002-02-01,0.1390,-2.844136,-2.076621,1.0098,1.201066,2000s
2,2002-03-01,0.1285,7.904382,3.673889,0.6827,5.582392,2000s
3,2002-04-01,0.1341,0.831264,-6.141762,-5.0435,1.523733,2000s
4,2002-05-01,0.1546,-4.543053,-0.908148,3.6252,0.533105,2000s


In [5]:
df_2000.columns

Index(['Date', 'U.S. 30 Day TBill TR', 'Russell 2000 TR', 'S&P 500 TR',
       'LB LT Gvt/Credit TR', 'MSCI EAFE TR', 'Period'],
      dtype='object')

In [33]:
df_2000[['U.S. 30 Day TBill TR', 'Russell 2000 TR', 'S&P 500 TR',
       'LB LT Gvt/Credit TR', 'MSCI EAFE TR']].std() * np.sqrt(12)

U.S. 30 Day TBill TR     0.489856
Russell 2000 TR         21.068986
S&P 500 TR              15.925935
LB LT Gvt/Credit TR     14.993268
MSCI EAFE TR            19.403445
dtype: float64

In [7]:
yearly_df = (
        df_2000.drop(["Period"], axis=1)
        .assign(
            Year=lambda x: x["Date"].apply(lambda x: x.split("-")[0]),
        )
        .drop("Date", axis=1)
        .groupby("Year")
        .sum()
        .reset_index()
        # .drop("Year", axis=1)
    )
yearly_df.head()

,Year,U.S. 30 Day TBill TR,Russell 2000 TR,S&P 500 TR,LB LT Gvt/Credit TR,MSCI EAFE TR
0,2002,1.6623,-21.568170,-24.351692,9.355600,-14.999577
1,2003,1.0383,39.168388,24.221238,4.729401,35.413283
2,2004,1.1099,16.773625,8.883624,7.170300,17.985116
3,2005,2.7751,4.231425,3.238415,9.425300,13.046591
4,2006,4.5961,16.638688,12.985788,5.416600,23.584398


In [17]:
from scipy.stats import gmean
def calculate_geometric_mean(dataframe):
    # Filter only numeric columns
    numeric_columns = dataframe.select_dtypes(include="number")

    # Calculate the geometric mean for each numeric column
    geometric_means = numeric_columns.apply(gmean, axis=0)

    return geometric_means

In [69]:
def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))-1

In [70]:
iterable = [i+1 for i in (yearly_df.drop("Year",axis=1)/100)["Russell 2000 TR"].tolist()]

In [72]:
iterable

[0.7843183019000001,
 1.391683875,
 1.1677362543,
 1.0423142538,
 1.166386878,
 0.9794480694,
 0.6200125298000001,
 1.26664169,
 1.254548025,
 0.9686021248]

In [71]:
(geo_mean(iterable))*100

3.7740431801760677

In [52]:
import pandas as pd

def calculate_geometric_returns(dataframe):
    """
    Calculate the geometric annual returns (CAGR) for each column (asset) in a DataFrame.

    Parameters:
    - dataframe: A Pandas DataFrame where each column represents a different asset's yearly means.

    Returns:
    - A Pandas Series containing the geometric annual returns for each asset.
    """
    geometric_returns = {}

    for column in dataframe.columns:
        series = dataframe[column]
        
        # Check if the series has at least two data points
        if len(series) < 2:
            raise ValueError(f"Column '{column}' does not have enough data points to calculate returns.")
        
        initial_value = series.iloc[0]
        final_value = series.iloc[-1]
        time_period = len(series) - 1  # Number of years
        
        # Calculate the CAGR formula
        cagr = ((final_value / initial_value) ** (1 / time_period)) - 1

        # Convert CAGR to a percentage
        cagr_percentage = cagr * 100

        geometric_returns[column] = cagr_percentage

    return pd.Series(geometric_returns, name="Geometric Returns")

calculate_geometric_returns(yearly_df.drop("Year",axis=1)/100)

/var/folders/h1/rvjbcw3x19dcwnv6qxb4pxrm0000gn/T/ipykernel_83442/2553112695.py:27: RuntimeWarning: invalid value encountered in double_scalars
  cagr = ((final_value / initial_value) ** (1 / time_period)) - 1


U.S. 30 Day TBill TR   -31.281924
Russell 2000 TR        -19.274701
S&P 500 TR                    NaN
LB LT Gvt/Credit TR     11.631494
MSCI EAFE TR            -3.194607
Name: Geometric Returns, dtype: float64

In [30]:
yearly_df.select_dtypes(include="number").columns.tolist()

['U.S. 30 Day TBill TR',
 'Russell 2000 TR',
 'S&P 500 TR',
 'LB LT Gvt/Credit TR',
 'MSCI EAFE TR']

In [38]:
df_2000[['U.S. 30 Day TBill TR', 'Russell 2000 TR', 'S&P 500 TR',
       'LB LT Gvt/Credit TR', 'MSCI EAFE TR']].mean()*12

U.S. 30 Day TBill TR    1.788840
Russell 2000 TR         6.416920
S&P 500 TR              2.195381
LB LT Gvt/Credit TR     9.020870
MSCI EAFE TR            6.450622
dtype: float64

In [36]:
yearly_df.drop("Year", axis=1).mean()

U.S. 30 Day TBill TR    1.788840
Russell 2000 TR         6.416920
S&P 500 TR              2.195381
LB LT Gvt/Credit TR     9.020870
MSCI EAFE TR            6.450622
dtype: float64

In [35]:
yearly_df.drop("Year", axis=1).std()

U.S. 30 Day TBill TR     1.731645
Russell 2000 TR         23.524448
S&P 500 TR              21.650257
LB LT Gvt/Credit TR      8.954625
MSCI EAFE TR            24.801192
dtype: float64

In [32]:
for i in ['U.S. 30 Day TBill TR',
 'Russell 2000 TR',
 'S&P 500 TR',
 'LB LT Gvt/Credit TR',
 'MSCI EAFE TR']:
    
    print(np.exp(yearly_df[i].apply(np.log).mean()))

0.8096632419455239
17.76142050156194
7.761932099510882
9.071893450000298
17.872236466613888


In [16]:
calculate_geometric_mean(yearly_df)

/Users/hanyuwu/Library/Caches/pypoetry/virtualenvs/asset-allocation-project-ax0yMTeZ-py3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:197: RuntimeWarning: invalid value encountered in log
  log_a = np.log(a)


0          NaN
1    11.053134
2     7.341593
3     5.419734
4    10.487348
5          NaN
6          NaN
7          NaN
8     4.821103
9          NaN
dtype: float64

In [33]:
(
    pd.melt(
        df_2000.drop("Period", axis=1),
        id_vars=["Date"],
        var_name="Indexes",
        value_name="Value",
    )
    .sort_values(by=["Date", "Indexes"])
    .groupby("Indexes")
    .agg(
        Monthly_average_return=("Value", "mean"),
        Monthly_sd=("Value", "std"),
    )
    .reset_index()
).merge(
    (
        pd.melt(
            yearly_df,
            id_vars=["Year"],
            var_name="Indexes",
            value_name="Value",
        )
        .sort_values(by=["Year", "Indexes"])
        .groupby("Indexes")
        .agg(
            Annualized_return=("Value", "mean"),
            Annualized_sd=("Value", "std"),
        )
        .reset_index()
    ),
    on="Indexes"
)

,Indexes,Monthly_average_return,Monthly_sd,Annualized_return,Annualized_sd
0,LB LT Gvt/Credit TR,0.751739,4.328184,9.020870,8.954625
1,MSCI EAFE TR,0.537552,5.601292,6.450622,24.801192
2,Russell 2000 TR,0.534743,6.082092,6.416920,23.524448
3,S&P 500 TR,0.182948,4.597421,2.195381,21.650257
4,U.S. 30 Day TBill TR,0.149070,0.141409,1.788840,1.731645


In [38]:
yearly_df.drop(["Year","U.S. 30 Day TBill TR"], axis=1).cov()

,Russell 2000 TR,S&P 500 TR,LB LT Gvt/Credit TR,MSCI EAFE TR
Russell 2000 TR,553.399671,489.712588,-144.029198,544.448239
S&P 500 TR,489.712588,468.733615,-128.824340,511.428691
LB LT Gvt/Credit TR,-144.029198,-128.824340,80.185302,-172.997660
MSCI EAFE TR,544.448239,511.428691,-172.997660,615.099102


In [18]:
(
    pd.melt(
        df_2000.drop("Period", axis=1),
        id_vars=["Date"],
        var_name="Indexes",
        value_name="Value",
    )
    .sort_values(by=["Date", "Indexes"])
    .groupby("Indexes")
    .agg(
        Monthly_average_return=("Value", "mean"),
        Monthly_sd=("Value", "std"),
    )
    .reset_index()
)
    # .reset_index()
    # .assign(Annualized_return=lambda x: x["Monthly_average_return"] * 12)
    # .assign(Annualized_sd=lambda x: x["Monthly_sd"] * 12**0.5)
    # )

,Indexes,Monthly_average_return,Monthly_sd
0,LB LT Gvt/Credit TR,0.751739,4.328184
1,MSCI EAFE TR,0.537552,5.601292
2,Russell 2000 TR,0.534743,6.082092
3,S&P 500 TR,0.182948,4.597421
4,U.S. 30 Day TBill TR,0.149070,0.141409


In [ ]:
def generate_monthly_n_annual_stats_df(data, time_range):
    monthly_annual_stats = (
        pd.melt(
            data.query("Period==@time_range").drop("Period", axis=1),
            id_vars=["Date"],
            var_name="Indexes",
            value_name="Value",
        )
        .sort_values(by=["Date", "Indexes"])
        .groupby("Indexes")
        .agg(
            Monthly_average_return=("Value", "mean"),
            Monthly_sd=("Value", "std"),
        )
        .reset_index()
        .assign(Annualized_return=lambda x: x["Monthly_average_return"] * 12)
        .assign(Annualized_sd=lambda x: x["Monthly_sd"] * 12**0.5)
    )

    st.dataframe(
        monthly_annual_stats.style.apply(highlight_max_by_column),
        use_container_width=True,
        hide_index=True,
    )